# Исследование надёжности заёмщиков

Заказчик — кредитный отдел банка. Необходимо разобраться, влияет ли семейное положение и количество детей клиента на факт погашения кредита в срок. Входные данные от банка — статистика о платёжеспособности клиентов.

Результаты исследования будут учтены при построении модели **кредитного скоринга** — специальной системы, которая оценивает способность потенциального заёмщика вернуть кредит банку.

<h1>Оглавление<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Обзор-данных" data-toc-modified-id="Обзор-данных-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Обзор данных</a></span></li><li><span><a href="#Предобработка-данных" data-toc-modified-id="Предобработка-данных-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Предобработка данных</a></span><ul class="toc-item"><li><span><a href="#Обработка-пропусков" data-toc-modified-id="Обработка-пропусков-2.1"><span class="toc-item-num">2.1&nbsp;&nbsp;</span>Обработка пропусков</a></span></li><li><span><a href="#Замена-типа-данных" data-toc-modified-id="Замена-типа-данных-2.2"><span class="toc-item-num">2.2&nbsp;&nbsp;</span>Замена типа данных</a></span></li><li><span><a href="#Обработка-дубликатов" data-toc-modified-id="Обработка-дубликатов-2.3"><span class="toc-item-num">2.3&nbsp;&nbsp;</span>Обработка дубликатов</a></span></li><li><span><a href="#Лемматизация" data-toc-modified-id="Лемматизация-2.4"><span class="toc-item-num">2.4&nbsp;&nbsp;</span>Лемматизация</a></span></li><li><span><a href="#Категоризация-данных" data-toc-modified-id="Категоризация-данных-2.5"><span class="toc-item-num">2.5&nbsp;&nbsp;</span>Категоризация данных</a></span></li></ul></li><li><span><a href="#Проверка-гипотез" data-toc-modified-id="Проверка-гипотез-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>Проверка гипотез</a></span></li><li><span><a href="#Общий-вывод" data-toc-modified-id="Общий-вывод-4"><span class="toc-item-num">4&nbsp;&nbsp;</span>Общий вывод</a></span></li><li><span><a href="#Чек-лист-готовности-проекта" data-toc-modified-id="Чек-лист-готовности-проекта-5"><span class="toc-item-num">5&nbsp;&nbsp;</span>Чек-лист готовности проекта</a></span></li></ul></div>

##  Обзор данных

Составим первое представление о входных данных от банка.

In [1]:
import pandas as pd
from pymystem3 import Mystem

In [2]:
df = pd.read_csv('/datasets/data.csv')

Выведем 10 первых строк таблицы:

In [3]:
df.head(10)

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,-8437.673028,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875.639453,покупка жилья
1,1,-4024.803754,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080.014102,приобретение автомобиля
2,0,-5623.422610,33,Среднее,1,женат / замужем,0,M,сотрудник,0,145885.952297,покупка жилья
3,3,-4124.747207,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628.550329,дополнительное образование
4,0,340266.072047,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616.077870,сыграть свадьбу
5,0,-926.185831,27,высшее,0,гражданский брак,1,M,компаньон,0,255763.565419,покупка жилья
6,0,-2879.202052,43,высшее,0,женат / замужем,0,F,компаньон,0,240525.971920,операции с жильем
7,0,-152.779569,50,СРЕДНЕЕ,1,женат / замужем,0,M,сотрудник,0,135823.934197,образование
8,2,-6929.865299,35,ВЫСШЕЕ,0,гражданский брак,1,F,сотрудник,0,95856.832424,на проведение свадьбы
9,0,-2188.756445,41,среднее,1,женат / замужем,0,M,сотрудник,0,144425.938277,покупка жилья для семьи


Выведем 10 последних строк таблицы:

In [4]:
df.tail(10)

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
21515,1,-467.685130,28,среднее,1,женат / замужем,0,F,сотрудник,1,109486.327999,заняться образованием
21516,0,-914.391429,42,высшее,0,женат / замужем,0,F,компаньон,0,322807.776603,покупка своего жилья
21517,0,-404.679034,42,высшее,0,гражданский брак,1,F,компаньон,0,178059.553491,на покупку своего автомобиля
21518,0,373995.710838,59,СРЕДНЕЕ,1,женат / замужем,0,F,пенсионер,0,153864.650328,сделка с автомобилем
21519,1,-2351.431934,37,ученая степень,4,в разводе,3,M,сотрудник,0,115949.039788,покупка коммерческой недвижимости
21520,1,-4529.316663,43,среднее,1,гражданский брак,1,F,компаньон,0,224791.862382,операции с жильем
21521,0,343937.404131,67,среднее,1,женат / замужем,0,F,пенсионер,0,155999.806512,сделка с автомобилем
21522,1,-2113.346888,38,среднее,1,гражданский брак,1,M,сотрудник,1,89672.561153,недвижимость
21523,3,-3112.481705,38,среднее,1,женат / замужем,0,M,сотрудник,1,244093.050500,на покупку своего автомобиля
21524,2,-1984.507589,40,среднее,1,женат / замужем,0,F,сотрудник,0,82047.418899,на покупку автомобиля


Выведем 10 случайных строк таблицы:

In [5]:
df.sample(10, random_state=1)

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
1383,0,353802.811675,37,среднее,1,вдовец / вдова,2,F,пенсионер,0,216452.226085,строительство недвижимости
300,1,-359.193975,33,СРЕДНЕЕ,1,гражданский брак,1,M,сотрудник,0,223001.623994,на проведение свадьбы
6565,2,-1064.854333,35,среднее,1,гражданский брак,1,F,компаньон,0,163591.209323,свадьба
17027,0,NaN,48,высшее,0,гражданский брак,1,F,сотрудник,0,NaN,операции с жильем
4077,0,-7059.100220,45,высшее,0,гражданский брак,1,F,компаньон,1,194820.185757,сыграть свадьбу
10437,0,-2258.301455,50,среднее,1,женат / замужем,0,F,сотрудник,0,146768.250599,операции с коммерческой недвижимостью
9631,1,-1045.752744,23,среднее,1,женат / замужем,0,M,сотрудник,0,81734.568425,операции с коммерческой недвижимостью
19867,0,NaN,39,среднее,1,женат / замужем,0,M,сотрудник,0,NaN,дополнительное образование
9124,0,-282.186487,48,среднее,1,в разводе,3,F,компаньон,0,284275.683334,строительство жилой недвижимости
19634,0,-1320.103813,46,среднее,1,женат / замужем,0,F,компаньон,0,184569.030590,автомобили


Выведем описательные статистики для числовых данные таблицы:

In [6]:
df.describe().T

,count,mean,std,min,25%,50%,75%,max
children,21525.0,0.538908,1.381587,-1.000000,0.000000,0.000000,1.000000,2.000000e+01
days_employed,19351.0,63046.497661,140827.311974,-18388.949901,-2747.423625,-1203.369529,-291.095954,4.017554e+05
dob_years,21525.0,43.293380,12.574584,0.000000,33.000000,42.000000,53.000000,7.500000e+01
education_id,21525.0,0.817236,0.548138,0.000000,1.000000,1.000000,1.000000,4.000000e+00
family_status_id,21525.0,0.972544,1.420324,0.000000,0.000000,0.000000,1.000000,4.000000e+00
debt,21525.0,0.080883,0.272661,0.000000,0.000000,0.000000,0.000000,1.000000e+00
total_income,19351.0,167422.302208,102971.566448,20667.263793,103053.152913,145017.937533,203435.067663,2.265604e+06


In [7]:
df.info(memory_usage='deep')

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
children            21525 non-null int64
days_employed       19351 non-null float64
dob_years           21525 non-null int64
education           21525 non-null object
education_id        21525 non-null int64
family_status       21525 non-null object
family_status_id    21525 non-null int64
gender              21525 non-null object
income_type         21525 non-null object
debt                21525 non-null int64
total_income        19351 non-null float64
purpose             21525 non-null object
dtypes: float64(2), int64(5), object(5)
memory usage: 11.3 MB


Согласно документации к данным:
* children — количество детей в семье
* days_employed — общий трудовой стаж в днях
* dob_years — возраст клиента в годах
* education — уровень образования клиента
* education_id — идентификатор уровня образования
* family_status — семейное положение
* family_status_id — идентификатор семейного положения
* gender — пол клиента
* income_type — тип занятости
* debt — имел ли задолженность по возврату кредитов
* total_income — ежемесячный доход
* debt — имел ли задолженность по возврату кредитов
* purpose — цель получения кредита

**Вывод**

Данная таблица содержит основную информацию о заёмщике. На первый взгляд можно выявить следующие ошибки в данных:
1. Отрицательные значения в столбце, содержащий данные об общем трудовом стаже в днях. (`data['days_employed']`)
2. Отсутствие единого стиля в столбце, содержащий данные об образовании. (`data['education']`)
3. Встречаются пропуски в данных.
4. Дробные значения в стоблце `data['days_employed']`

Предварительно можно утверждать, что, данных достаточно для проверки гипотез.
Для дальнейшего исследования необходимо исправить существующие ошибки, описать природу пропущенных значений и выбрать наиболее подходящий способ их заполнения, проверить данные на наличие дубликатов, а также изменить тип данных там, где это необходимо.

## Предобработка данных

### Обработка пропусков

Создадим функцию `missed_values`, которая возвращает количество пропусков в данных:

In [8]:
def missed_values(data):
    return data.isna().sum().to_frame('missed_values')

Рассчитаем количество пропущенных значений в каждом столбце:

In [9]:
missed_values(df)

,missed_values
children,0
days_employed,2174
dob_years,0
education,0
education_id,0
family_status,0
family_status_id,0
gender,0
income_type,0
debt,0


Рассмотрим первые 10 строк, в которых присутствуют пропущенные значения:

In [10]:
df[df['total_income'].isna()].head(10)

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
12,0,NaN,65,среднее,1,гражданский брак,1,M,пенсионер,0,NaN,сыграть свадьбу
26,0,NaN,41,среднее,1,женат / замужем,0,M,госслужащий,0,NaN,образование
29,0,NaN,63,среднее,1,Не женат / не замужем,4,F,пенсионер,0,NaN,строительство жилой недвижимости
41,0,NaN,50,среднее,1,женат / замужем,0,F,госслужащий,0,NaN,сделка с подержанным автомобилем
55,0,NaN,54,среднее,1,гражданский брак,1,F,пенсионер,1,NaN,сыграть свадьбу
65,0,NaN,21,среднее,1,Не женат / не замужем,4,M,компаньон,0,NaN,операции с коммерческой недвижимостью
67,0,NaN,52,высшее,0,женат / замужем,0,F,пенсионер,0,NaN,покупка жилья для семьи
72,1,NaN,32,высшее,0,женат / замужем,0,M,госслужащий,0,NaN,операции с коммерческой недвижимостью
82,2,NaN,50,высшее,0,женат / замужем,0,F,сотрудник,0,NaN,жилье
83,0,NaN,52,среднее,1,женат / замужем,0,M,сотрудник,0,NaN,жилье


Пропущенные значения попарно находятся в столбцах `days_employed` и `total_income`.



Рассмотрим 10 случайных строк столбца `days_employed`:

In [11]:
df['days_employed'].sample(10, random_state=1).to_frame()

,days_employed
1383,353802.811675
300,-359.193975
6565,-1064.854333
17027,NaN
4077,-7059.100220
10437,-2258.301455
9631,-1045.752744
19867,NaN
9124,-282.186487
19634,-1320.103813


Заметим, что в данных, содержащий общее количество трудового стажа в днях присутствуют отрицательны значения. Однако количество дней не может быть отрицательной величиной.

Всего таких строк в датафрейме:

In [12]:
len(df[df['days_employed'] < 0])

15906

Исправим данную ошибку в данных:

In [13]:
df['days_employed'] = df['days_employed'].apply(abs)

Заменим пропущенные значения ежемесячного дохода и общего трудового стажа медианным значением.

In [14]:
median_days_employed = df['days_employed'].median()
print(f'Медианное значение трудового стажа: {median_days_employed:.0f}')

Медианное значение трудового стажа: 2194


In [15]:
median_income = df['total_income'].median()
print(f'Медианное значение ежемесячного дохода: {median_income:.2f}')

Медианное значение ежемесячного дохода: 145017.94


In [16]:
df['total_income'] = df['total_income'].fillna(median_income)
df['days_employed'] = df['days_employed'].fillna(median_days_employed)

Убедимся, что в таблице не осталось пропусков:

In [17]:
missed_values(df)

,missed_values
children,0
days_employed,0
dob_years,0
education,0
education_id,0
family_status,0
family_status_id,0
gender,0
income_type,0
debt,0


**Вывод**

Все пропущенные значения попарно находились в стоблцах `days_employed` и `total_income`, описывающие общий трудовой стаж и ежемесячный доход. 

Предположительно, данные были пропущены из-за того, что данные пострадали при сборке.

Отсутствующие данные были заполнены медианными значениями ежемесячного дохода и трудового стажа.

Так как среднее значение может значительно исказить представление о данных (минимальные и максимальные значения могут быть как сильно больше среднего, так и сильно меньше), то для обработки пропусков были выбраны именно медианные значения.

### Замена типа данных

Приведем значения в столбцах `days_employed` и `total_income` к целочисленному типу `int`.
Для этого воспользуемся методом `astype('int')`

Создадим функцию `mem_usage`, возвращающая количество памяти в МБ, которую занимает датафрейм:

In [18]:
def mem_usage(pandas_obj):
    if isinstance(pandas_obj, pd.DataFrame):
        usage_b = pandas_obj.memory_usage(deep=True).sum()
    else: # исходим из предположения о том, что если это не DataFrame, то это Series
        usage_b = pandas_obj.memory_usage(deep=True)
    usage_mb = usage_b / 1024 ** 2 # преобразуем байты в мегабайты
    return "{:03.2f}".format(usage_mb)

Определим переменную `mem_before`, показывающее, какое количество памяти занимает датафрейм до преобразования типов.

In [19]:
mem_before = mem_usage(df)
print(f'{mem_before} МБ')

11.28 МБ


Заменим тип данных в столбцах `days_employed` и `total_income` из `float` в `uint32`:

In [20]:
df[['days_employed', 'total_income']] = df[['days_employed', 'total_income']].astype('uint32')

Заменим тип данных в столбцах `children`, `dob_years`, `education_id`, `family_status_id`, `debt` из `int` в `uint8`:

In [21]:
df[['children', 'dob_years', 'education_id', 'family_status_id', 'debt']] = \
df[['children', 'dob_years', 'education_id', 'family_status_id', 'debt']].astype('uint8')

Заменим тип данных в столбцах `education`, `family_status`, `income_type`, `purpose`, `gender` из `object` в `category`:

In [22]:
df[['education', 'family_status', 'income_type', 'purpose', 'gender']] = \
df[['education', 'family_status', 'income_type', 'purpose', 'gender']].astype('category')

Определим переменную `mem_before`, показывающее, какое количество памяти занимает датафрейм до преобразования типов.

In [23]:
mem_after = mem_usage(df)
print(f'{mem_after} МБ')

0.38 МБ


In [24]:
saved_memory = float(mem_after) / float(mem_before)
print(f'После замены типа данных удалось сэкономить {1 - saved_memory:.0%} памяти.')

После замены типа данных удалось сэкономить 97% памяти.


**Вывод**

Для оптимизации используемой памяти, первоначальные типы данных в зависимости от того, что в них хранится, были заменены на другие. Были использованы типы `uint8`, `uint32`, `category`.

В результате замены удалось сократить количество используемой памяти на 97%.

Для замены типа данных в `pandas` существует несколько методов: `to_numeric()` и `astype()`.

Метод `to_numeric()` возвращает тип `float`. 

### Обработка дубликатов

В первую очередь приведем значения в столбце `education` к единому стилю. Это позволит выявить точное количество явных дубликатов. Для этого воспользуемся методом `str.lower()`:

In [25]:
df['education'] = df['education'].str.lower()

Создадим функцию, возвращающую количество дубликатов в данных:

In [26]:
def duplicates(data):
    return data.duplicated().sum()

Рассчитаем количество дубликатов:

In [27]:
duplicates(df)

71

Избавимся от явных дубликатов:

In [28]:
df = df.drop_duplicates().reset_index(drop=True)

Убедимся, что полностью избавились от явных дубликатов:

In [29]:
duplicates(df)

0

**Вывод**

Причиной появления дубликатов в данных может быть несколько:
* Ошибка при вводе данных о заемщике.
* Ошибка при отправке заявки на кредит от заемщика: одна и та же заявка могла быть отправлена несколько раз.

Для обработки явных дубликатов, столбец `education` был приведен к одному стилю заполнения, что позволило обнаружить все дубликаты. Было рассчитано количество дубликатов, которые по итогу были удалены, при помощи метода `drop_duplicates()`.

### Лемматизация

Для проведения леммантизации, воспользуемся библиотекой `pymystem3`:

In [30]:
m = Mystem()

Рассмотрим все уникальные значения столбца `purpose`:

In [31]:
purpose_unique = list(df['purpose'].unique())
purpose_unique

['покупка жилья',
 'приобретение автомобиля',
 'дополнительное образование',
 'сыграть свадьбу',
 'операции с жильем',
 'образование',
 'на проведение свадьбы',
 'покупка жилья для семьи',
 'покупка недвижимости',
 'покупка коммерческой недвижимости',
 'покупка жилой недвижимости',
 'строительство собственной недвижимости',
 'недвижимость',
 'строительство недвижимости',
 'на покупку подержанного автомобиля',
 'на покупку своего автомобиля',
 'операции с коммерческой недвижимостью',
 'строительство жилой недвижимости',
 'жилье',
 'операции со своей недвижимостью',
 'автомобили',
 'заняться образованием',
 'сделка с подержанным автомобилем',
 'получение образования',
 'автомобиль',
 'свадьба',
 'получение дополнительного образования',
 'покупка своего жилья',
 'операции с недвижимостью',
 'получение высшего образования',
 'свой автомобиль',
 'сделка с автомобилем',
 'профильное образование',
 'высшее образование',
 'покупка жилья для сдачи',
 'на покупку автомобиля',
 'ремонт жилью',
 '

Получим список леммантизированных слов для списка, содержащий уникальные значения `purpose`:

In [32]:
lemmas = [m.lemmatize(purpose) for purpose in purpose_unique]

In [33]:
lemmas

[['покупка', ' ', 'жилье', '\n'],
 ['приобретение', ' ', 'автомобиль', '\n'],
 ['дополнительный', ' ', 'образование', '\n'],
 ['сыграть', ' ', 'свадьба', '\n'],
 ['операция', ' ', 'с', ' ', 'жилье', '\n'],
 ['образование', '\n'],
 ['на', ' ', 'проведение', ' ', 'свадьба', '\n'],
 ['покупка', ' ', 'жилье', ' ', 'для', ' ', 'семья', '\n'],
 ['покупка', ' ', 'недвижимость', '\n'],
 ['покупка', ' ', 'коммерческий', ' ', 'недвижимость', '\n'],
 ['покупка', ' ', 'жилой', ' ', 'недвижимость', '\n'],
 ['строительство', ' ', 'собственный', ' ', 'недвижимость', '\n'],
 ['недвижимость', '\n'],
 ['строительство', ' ', 'недвижимость', '\n'],
 ['на', ' ', 'покупка', ' ', 'подержать', ' ', 'автомобиль', '\n'],
 ['на', ' ', 'покупка', ' ', 'свой', ' ', 'автомобиль', '\n'],
 ['операция', ' ', 'с', ' ', 'коммерческий', ' ', 'недвижимость', '\n'],
 ['строительство', ' ', 'жилой', ' ', 'недвижимость', '\n'],
 ['жилье', '\n'],
 ['операция', ' ', 'со', ' ', 'свой', ' ', 'недвижимость', '\n'],
 ['автомобиль'

**Вывод**

Были проведен анализ уникальных значений целей получения кредита. Для этого был использован метод леммантизации при помощи библиотеки `pymystem3`. Исходя из полученных результатов следует, что уникальные значения по итогу сводятся всего к 4-м категориям:
1. Проведение свадьбы
2. Покупка недвижимости
3. Покупка автомобиля
4. Получения образования

### Категоризация данных

Выясним, сколько раз встречается каждая цель получения кредита:

In [34]:
df['purpose'].value_counts()

свадьба                                   791
на проведение свадьбы                     768
сыграть свадьбу                           765
операции с недвижимостью                  675
покупка коммерческой недвижимости         661
операции с жильем                         652
покупка жилья для сдачи                   651
операции с коммерческой недвижимостью     650
жилье                                     646
покупка жилья                             646
покупка жилья для семьи                   638
строительство собственной недвижимости    635
недвижимость                              633
операции со своей недвижимостью           627
строительство жилой недвижимости          624
покупка недвижимости                      621
покупка своего жилья                      620
строительство недвижимости                619
ремонт жилью                              607
покупка жилой недвижимости                606
на покупку своего автомобиля              505
заняться высшим образованием      

Для категоризации данных по целям получения кредита создадим функцию `categorized_purpose`:

In [35]:
def categorized_purpose(purpose):
    lemmas = m.lemmatize(purpose)
    for lemma in lemmas:
        if 'авто' in lemma:
            return 'Покупка автомобиля'
        if 'свад' in lemma:
            return 'Проведение свадьбы'
        if 'образ' in lemma:
            return 'Получение образования'
        if ('жил' in lemma) or ('недвиж' in lemma):
            return 'Покупка недвижимости'


Заменим исходные данные в столбце `purpose` на полученные категориальные. Для этого воспользуемся методом `apply`, которые применит функцию `categorized_purpose` к каждому значению в столбце `purpose`. 

In [36]:
df['purpose'] = df['purpose'].apply(categorized_purpose) 

Рассмотрим цели получения кредита, разбитые по категориям:

In [37]:
df['purpose'].value_counts()

Покупка недвижимости     10811
Покупка автомобиля        4306
Получение образования     4013
Проведение свадьбы        2324
Name: purpose, dtype: int64

Так как данные были упорядочены по категориям, то в данных могут появиться новые дубликаты, которые прежде было невозможно выявить. Проверим данную гипотезу:

In [38]:
duplicates(df)

334

В данных образовалось некоторое количество явных дубликатов. Обработаем их:

In [39]:
df = df.drop_duplicates().reset_index(drop=True)

Убедимся, что дубликатов больше нет:

In [40]:
duplicates(df)

0

**Вывод**

Таким образом цели получения кредита свелись всего к 4 - м категориям. Наибольшее количество людей хотят получить кредит с целью покупки жилья или недвижимости.

Была создана функция `categorized_purpose`, которая заменяет исходные значения столбца `purpose` к категоризированным.

Также были устранены дубликаты, наличие которых невозможно было обнаружить без категоризации.

## Проверка гипотез

- Есть ли зависимость между наличием детей и возвратом кредита в срок?

Для ответа на вопросы учитываем, что если значение debt = 1, то заемщик имел задолженность по кредиту ранее. Если debt = 0, то задолженность ранее отсутствовала.

Создадим функцию `children_id`, которая возвращает 1, если у заемщика есть хотя бы 1 ребенок и 0, если детей нет:

In [41]:
def children_id(child):
    if child != 0:
        return 1
    return 0

Создадим столбец `children_id`:

In [42]:
df['children_id'] = df['children'].apply(children_id)

Построим сводную таблицу:

In [43]:
df.pivot_table(['debt'], 'children_id', aggfunc='mean').sort_values(by='debt', ascending=False)

,debt
children_id,
1,0.092991
0,0.076723


**Вывод**

Была построена сводная таблица при помощи метода `pivot_table`, отражающая зависимость между наличием детей и возвратом кредита в срок. Из таблицы видно, что процент заемщиков, у которых отсутствуют дети, меньше, нежели чем процент заемщиков, у кого дети есть.

- Есть ли зависимость между семейным положением и возвратом кредита в срок?

In [44]:
df.pivot_table(['debt'], 'family_status', aggfunc='mean').sort_values(by='debt', ascending=False)

,debt
family_status,
Не женат / не замужем,0.098420
гражданский брак,0.094083
женат / замужем,0.076929
в разводе,0.071249
вдовец / вдова,0.066808


**Вывод**

Из таблицы видно: люди, не состоящие в официальных отношениях, чаще не возвращают в кредит в срок, в отличие от тех, кто состоит или ранее состоял в браке.

- Есть ли зависимость между уровнем дохода и возвратом кредита в срок?

Разобьем ежемесячный доход на 5 интервалов при помощи метода `qcut`:

In [45]:
df['total_income_group'] = pd.qcut(df['total_income'], 5)

In [46]:
df.pivot_table(['debt'], 'total_income_group', aggfunc='mean').sort_values(by='total_income_group', ascending=False)

,debt
total_income_group,
"(215777.2, 2265604.0]",0.070312
"(157730.2, 215777.2]",0.085464
"(134253.6, 157730.2]",0.093513
"(97949.4, 134253.6]",0.081913
"(20666.999, 97949.4]",0.080492


**Вывод**

Для построения сводной таблицы был использован метод `qcut`, позволяющий разбить доходы по некоторым интервалам.

Из таблицы видно, что данные распределены неравномерно. Это не позволяет определить явную зависимость между уровнем дохода и возвратом кредита в срок.

- Как разные цели кредита влияют на его возврат в срок?

In [47]:
df.pivot_table(['debt'], 'purpose', aggfunc='mean').sort_values(by='debt', ascending=False)

,debt
purpose,
Покупка автомобиля,0.094101
Получение образования,0.093340
Проведение свадьбы,0.080659
Покупка недвижимости,0.073832


**Вывод**

Заемщики, которые берут кредит для покупки недвижимости реже всего имеют задолженность. В свою очередь, заемщики, взявшие кредит для покупки автомобиля, наиболее часто не отдают долг вовремя.

## Общий вывод

В ходе исследования для проверки основных гипотез, данные в первую очередь были подвергнуты предобработке, которая состояла из:
* обработки дубликатов;
* обработки пропусков;
* леммантизации;
* категоризации данных.
   
Исследование показало, что около половины заемщиков берут кредит с целью покупки недвижимости. Также всего `7.4%` заемщиков с данной целью имели задолженность в прошлом. Вторая по популярности цель кредита - покупка автомобиля. Среди тех, кто покупает автомобиль в кредит также установлено максимально количество просрочек - `9.4%` заемщиков имели задолженность.

Было выяснено, что наличие детей в худшую сторону влияет на платежеспособность заемщика. Среди заемщиков без детей `7.7%` имели долг по кредиту в прошлом. Но уже наличие одного ребенка в семье поднимает число должников до `9.3%`. Так как мы рассматриваем довольно большой объем данных, то повышение даже на `1.6%` является большим. Это неудивительно, так как ребенок требует немалых финансовых вложений.

Процент заемщиков, находящихся или находившихся в браке, реже имеют просрочку по кредиту - значение варируется в пределах `6.7 - 7.7%`, в отличие от холостых или же не состоящих в официальных отношениях людей: процент должников более `9%`. Данное наблюдение может быть связано с тем, что обычно в браке работают оба партнера. Следовательно, погашать кредит проще. 